In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ACER\\MLFLow_DVC_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ACER\\MLFLow_DVC_Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager: 
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} has been downloaded with following info: \n {headers}")
        else: 
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise(e)

[2024-03-07 16:49:27,355: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-07 16:49:27,357: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-07 16:49:27,359: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-07 16:49:27,360: INFO: common: Directory created at: artifacts]
[2024-03-07 16:49:27,361: INFO: common: Directory created at: artifacts/data_ingestion]
[2024-03-07 16:49:27,363: INFO: 1474630251: File already exists of size: ~ 23 KB]
